## LDA（Latent Dirichlet Allocation）

トピックモデルと呼ばれる手法の一つ。文書集合に潜在するトピックを推定し、それぞれの文書が持つトピックの割合の結果を用いてクラスタリングする。  
文書が作られる過程として、  

「文書のトピックが決まり、それによって単語が決まる」  

というものを仮定する。  
上記の過程の全体を確立分布としてモデル化し、その確率分布のパラメータをデータから推定していく。

### ライブラリのインストール

In [3]:
# gemsimライブラリのインストール、統計的な計算を用いる自然言語処理に便利な機能を提供している
!pip3 install gensim

import itertools
import json
import logging
import math

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

# https://www.sejuku.net/blog/66459
# システムに関する処理をまとめたライブラリのsysを読み込む
import sys
# 下記でライブラリを読み込めるパス一覧を表示できる。ここにパスを書き込むと異なる階層からライブラリを読み込む事が可能となる。
print(sys.path)
# sys.path.append("相対パス")でsys.pathに追加、ここではディレクトリまでを指定する
sys.path.append("src")

from annoutil import find_xs_in_y
import sqlitedatastore as datastore

    100% |████████████████████████████████| 24.2MB 71kB/s eta 0:00:011  3% |█                               | 839kB 6.1MB/s eta 0:00:04    29% |█████████▋                      | 7.3MB 7.7MB/s eta 0:00:03    39% |████████████▌                   | 9.5MB 6.2MB/s eta 0:00:03    45% |██████████████▊                 | 11.1MB 6.8MB/s eta 0:00:02    80% |█████████████████████████▋      | 19.4MB 8.7MB/s eta 0:00:01
    100% |████████████████████████████████| 20.0MB 88kB/s  eta 0:00:011 7% |██▎                             | 1.5MB 7.5MB/s eta 0:00:03    20% |██████▌                         | 4.0MB 3.3MB/s eta 0:00:05    53% |█████████████████               | 10.7MB 10.2MB/s eta 0:00:01    59% |███████████████████             | 11.8MB 8.4MB/s eta 0:00:01    88% |████████████████████████████▎   | 17.7MB 24.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://

### LDAの計算過程を保存

In [6]:
# LDAの計算過程を表示する為に、loggingモジュールでログの表示設定をする
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### クラスタリングに使用する文章の準備

In [15]:
datastore.connect()
sentences = []
# コーパスとして文ごとに単語の原型のリストをsents変数に格納する
for doc_id in datastore.get_all_ids(limit=-1):
    all_tokens = datastore.get_annotation(doc_id, 'token')
    for sent in datastore.get_annotation(doc_id, 'sentence'):
        tokens = find_xs_in_y(all_tokens, sent)
        # 固有表現を除くためにNE属性がOである単語の原型リストをsentences変数に格納する
        sentences.append([token['lemma'] for token in tokens if token.get('NE') == 'O'])

# 記事の数が少ないため、記事を分割して20文を1つの文章として扱う。
n_sent = 20
# 20文ごとにsentencesの中身を結合してdocs変数に格納する
# chain.from_iterable(['ABC', 'DEF']) --> A B C D E Fさらにリスト化している。
docs = [list(itertools.chain.from_iterable(sentences[i:i+n_sent])) for i in range(0, len(sentences), n_sent)]

dictionary = Dictionary(docs)
# filter_extremesで使用する単語を決める。
# no_below=2で2つ未満の文書にしか出現しない単語がはじかれる。
# no_above=0.3で全文書のうち30%以上の文書に出現する単語がはじかれる。
dictionary.filter_extremes(no_below=2, no_above=0.3)
# 各文書をLDAの入力とする単語の集まりに変換
corpus = [dictionary.doc2bow(doc) for doc in docs]
print(corpus[:3])
datastore.close()

2019-10-23 03:21:19,050 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-10-23 03:21:19,856 : INFO : built Dictionary(18125 unique tokens: ['*', 'β', '、', '「', '」']...) from 2679 documents (total 1197722 corpus positions)
2019-10-23 03:21:19,913 : INFO : discarding 6229 tokens: [('*', 2679), ('β', 1), ('、', 2654), ('「', 1410), ('」', 1405), ('ある', 2528), ('いる', 2477), ('おる', 1286), ('から', 2223), ('が', 2595)]...
2019-10-23 03:21:19,914 : INFO : keeping 11896 tokens which were in no less than 2 and no more than 803 (=30.0%) documents
2019-10-23 03:21:19,930 : INFO : resulting dictionary: Dictionary(11896 unique tokens: ['かかわる', 'かつて', 'くい', 'こい', 'さ']...)


[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 3), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 3), (65, 1), (66, 2), (67, 1), (68, 1), (69, 3), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 3), (78, 1), (79, 2), (80, 1), (81, 2), (82, 1), (83, 1)], [(13, 2), (20, 1), (22, 1), (23, 1), (24, 1), (34, 1), (37, 1), (46, 2), (58, 1), (69, 3), (77, 1), (81, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 2), (95, 2), (96, 2), (97, 1), (98, 1), (99, 2)

### LDAの実行

In [21]:
datastore.connect()

lda = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=10)
'''
LdaModelライブラリ
num_topicsでトピックの数を10にする
passesで同じ文書を10回学習するように設定
show_topics関数を使うためにid2word=dictionaryを設定しておく。
'''

# 主題の確認、show_topics関数でそれぞれのトピックの単語の確率分布として確率値の大きな単語から上位10個を表示するようにする。
for topic in lda.show_topics(num_topics=-1, num_words=10):
    print(f'topic id:{topic[0]:d}, words={topic[1]:s}')
    
# 記事の主題分布の推定
for doc_id in datastore.get_all_ids(limit=-1):
    meta_info = json.loads(datastore.get(doc_id, ['meta_info'])['meta_info'])
    title = meta_info['title']
    print(title)
    
    doc = [token['lemma'] for token in datastore.get_annotation(doc_id, 'token') if token.get('NE') == 'O']
    # get_document_topics関数で記事のトピック分布を推定し、結果を表示する
    for topic in sorted(lda.get_document_topics(dictionary.doc2bow(doc)), key=lambda x: x[1], reverse=True):
        print(f'\ttopic id:{topic[0]:d}, prob={topic[1]:f}')
        
datastore.close()

2019-10-23 04:04:31,372 : INFO : using symmetric alpha at 0.1
2019-10-23 04:04:31,373 : INFO : using symmetric eta at 0.1
2019-10-23 04:04:31,378 : INFO : using serial LDA version on this node
2019-10-23 04:04:31,402 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 2679 documents, updating model once every 2000 documents, evaluating perplexity every 2679 documents, iterating 50x with a convergence threshold of 0.001000
2019-10-23 04:04:31,403 : INFO : PROGRESS: pass 0, at document #2000/2679
2019-10-23 04:04:32,915 : INFO : merging changes from 2000 documents into a model of 2679 documents
2019-10-23 04:04:32,924 : INFO : topic #1 (0.100): 0.014*"大統領" + 0.006*"選挙" + 0.005*"政権" + 0.005*"系" + 0.004*"万" + 0.004*"政治" + 0.003*"関係" + 0.003*"民族" + 0.003*"月" + 0.003*"制"
2019-10-23 04:04:32,925 : INFO : topic #2 (0.100): 0.004*"派" + 0.004*"教育" + 0.004*"関係" + 0.003*"系" + 0.003*"表記" + 0.003*"政権" + 0.003*"間" + 0.003*"文化" + 0.003*"地" + 0.003*"量"
20

topic id:0, words=0.025*"輸出" + 0.019*"生産" + 0.015*"万" + 0.013*"量" + 0.012*"農業" + 0.012*"輸入" + 0.011*"工業" + 0.011*"トン" + 0.010*"位" + 0.010*"占める"
topic id:1, words=0.048*"大統領" + 0.029*"選挙" + 0.016*"首相" + 0.014*"議会" + 0.014*"制" + 0.014*"政権" + 0.013*"党" + 0.011*"議席" + 0.011*"政党" + 0.009*"選出"
topic id:2, words=0.011*"企業" + 0.011*"率" + 0.009*"成長" + 0.008*"産業" + 0.008*"位" + 0.007*"投資" + 0.007*"ドル" + 0.006*"高い" + 0.006*"金融" + 0.006*"通貨"
topic id:3, words=0.017*"王国" + 0.015*"帝国" + 0.011*"表記" + 0.011*"朝" + 0.010*"世" + 0.009*"領" + 0.009*"地" + 0.008*"国名" + 0.007*"支配" + 0.007*"歴史"
topic id:4, words=0.018*"文化" + 0.015*"教育" + 0.013*"『" + 0.013*"』" + 0.011*"音楽" + 0.008*"大学" + 0.008*"遺産" + 0.007*"文学" + 0.006*"料理" + 0.006*"歳"
topic id:5, words=0.017*"関係" + 0.009*"軍事" + 0.007*"国際" + 0.006*"加盟" + 0.005*"権" + 0.005*"連邦" + 0.005*"外交" + 0.005*"諸国" + 0.004*"問題" + 0.004*"機関"
topic id:6, words=0.015*"案内" + 0.015*"ツール" + 0.014*"遺産" + 0.011*"スポーツ" + 0.011*"その他" + 0.010*"サッカー" + 0.010*"選手" + 0.009*"共和" + 0.009*"大会

以上のようにトピックわけをすることでクラスタリングが可能となる。